# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770411898880                   -0.53    9.0         
  2   -2.771689648012       -2.89       -1.32    1.0    135ms
  3   -2.771714477130       -4.61       -2.51    1.0    155ms
  4   -2.771714666178       -6.72       -3.32    1.0    138ms
  5   -2.771714712535       -7.33       -3.82    2.0    164ms
  6   -2.771714715178       -8.58       -5.09    1.0    140ms
  7   -2.771714715247      -10.16       -5.30    2.0    167ms
  8   -2.771714715250      -11.66       -5.99    2.0    162ms
  9   -2.771714715250      -13.58       -6.87    1.0    156ms
 10   -2.771714715250   +  -13.92       -7.38    1.0    155ms
 11   -2.771714715250      -13.91       -8.08    1.0    171ms


-0.0001345727746860011

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770483939011                   -0.52    9.0         
  2   -2.771777942707       -2.89       -1.33    1.0    162ms
  3   -2.771801538489       -4.63       -2.42    1.0    138ms
  4   -2.771801986444       -6.35       -3.10    1.0    156ms
  5   -2.771802074391       -7.06       -4.58    2.0    180ms
  6   -2.771802074442      -10.29       -4.61    2.0    159ms
  7   -2.771802074475      -10.48       -5.64    1.0    164ms
  8   -2.771802074476      -11.97       -5.90    2.0    164ms
  9   -2.771802074476      -13.45       -6.60    1.0    170ms
 10   -2.771802074476      -13.73       -7.07    2.0    173ms
 11   -2.771802074476   +  -13.80       -7.61    1.0    162ms
 12   -2.771802074476      -14.51       -7.88    1.0    156ms
 13   -2.771802074476   +  -14.40       -9.21    2.0    179ms


0.017612221277773264

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345727746860011
Displaced dipole:  0.017612221277773264
Polarizability :   1.7746794052459265


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920760022e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551289033e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852761907740e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694594478218e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088789172430e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.289508847559e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.162164298309e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.162164298309e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 9.389015317957e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 6.756087519209e-12
[ Info: GMRES linsolve in iter 2; step 3: normres = 3.204691657107e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 3.204691657107e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 3.2047859175560343e-13
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.